# AWS Public Blockchain - Cross-Chain Examples

In [2]:
pip install PyAthena

Note: you may need to restart the kernel to use updated packages.


In [3]:
import boto3
import pandas as pd
from pyathena import connect

In [4]:
S3_BUCKET=!(aws cloudformation list-exports --query "Exports[?Name=='public-blockchain-athena-results'].Value" --output text)
S3_BUCKET=S3_BUCKET[0]

In [5]:
conn = connect(s3_staging_dir='s3://'+S3_BUCKET+'/results/')

### Get number of blocks per chain

In [6]:
df = pd.read_sql("select 'BTC' as chain,count(*) as block_count from btc.blocks group by 'BTC' union select 'ETH' as chain,count(*) as block_count from eth.blocks group by 'ETH'", conn)
df

,chain,block_count
0,BTC,754386
